**Assignment 4: Adverserial Search (Given: 28 Feb 2023, Due: 19 Mar 2023)**


**General instructions**

* Solutions are to be typed in the `.ipynb` file provided and uploaded in the lab course page in Moodle on the due date. 
* Your code should be well commented and should be compatible with python3.
* For this assignment, you are allowed to import the libraries `random` and `copy` of python3. No other libraries may be imported.


# Adverserial search

The Tic-Tac-Toe game starts on a 3x3 grid with two players "X" and "O" who take turns and play. The rules are as follows: each player gets a turn with player "X" (resp. "O") writing an "X" (resp. "O") in an empty cell of the grid. The game starts with the move of the "O" player. The first player to write on three horizontal or vertical or diagonal cells wins.

In [15]:
import copy
import random as rnd


In [16]:

class TicTacToe_Grid_State:
  def __init__(self,grid_list: list[list],m = 3) -> None:
    # [[' ',' ',' '],[' ',' ',' '],[' ',' ',' ']]
    self.grid_side_length = m
    self.grid_list = copy.deepcopy(grid_list)
    # its a list of list of strings (which is immutable)
    # self.vacant_position_indices = self.find_num_of_vacant_positions()
    self.parent = None
    self.utility = -10


  
  def find_num_of_vacant_positions(self):
    m = self.grid_side_length
    vacant_list = list()
    for i in range(m):
      for j in range(m):
        if self.grid_list[i][j] == ' ':
          vacant_list.append((i,j))
    return list(vacant_list)
  
  def print_current_grid_state(self):
    # print()
    # for row in self.grid_list:
    #   print(row)
    # print()
    side_length = 2*self.grid_side_length - 1
    grid_img =[
      list([' ' for _ in range(side_length)]) for _ in range(side_length)
    ]
    for i in range(side_length):
      if i%2 != 0:
        for j in range(side_length):
          if j%2 != 0:
            grid_img[i][j] = '+'
          else:
            grid_img[i][j] = '-'
      else:
        for j in range(side_length):
          if j%2 != 0:
            grid_img[i][j] = '|'
          else:
            i_in_actual_grid = int(i/2)
            j_in_actual_grid = int(j/2)
            grid_img[i][j] = self.grid_list[i_in_actual_grid][j_in_actual_grid]
    
    for i in range(side_length):
      for j in range(side_length):
        print(grid_img[i][j],end='')
      print()
    del grid_img

  # def __eq__(self, __o: object) -> bool:
  #   pass

  def __eq__(self, other) -> bool:
    return isinstance(other,TicTacToe_Grid_State) and self.grid_list == other.grid_list
  def __lt__(self,other) -> bool:
    return isinstance(other,TicTacToe_Grid_State) and True



In [17]:



class Agent:
  def __init__(self,m = 3) -> None:
    self.length_of_tic_tac_toe = m
    self.current_state = self.initial_state_of_every_game()
    self.depth = 0
    self.depth_count = 0
    self.number_of_leaves_of_the_game_tree_explored = 0
    self.number_of_leaves_of_the_game_tree_explored_for_minimax = 0
    self.number_of_leaves_of_the_game_tree_explored_for_alpha_beta = 0
    self.alpha = -10
    # highest value before searching 
    self.beta = 10
    # lowest value before searching

    self.max_depth_minimax = 0
    self.max_depth_alpha_beta = 0

  def initial_state_of_every_game(self) -> TicTacToe_Grid_State:
    # initial state before game
    grid_list = [
      [' ' for _ in range(self.length_of_tic_tac_toe)] for _ in range(self.length_of_tic_tac_toe)
    ]
    return TicTacToe_Grid_State(grid_list)


  def child_generator(self,current_state: TicTacToe_Grid_State, current_player_max = True):
    vacant_pos_list = current_state.find_num_of_vacant_positions()
    # generating child states from the current state
    children_states = [
      TicTacToe_Grid_State(current_state.grid_list) for _ in range(len(vacant_pos_list))
    ]
    if current_player_max:
      # if the player to play is max player, we have to put 'X'
      for i in range(len(vacant_pos_list)):
        (x,y) = vacant_pos_list[i]
        child_state = children_states[i]
        child_state.grid_list[x][y] = 'X'
    else:
      # if the player is min player, we have to put 'O'
      for i in range(len(vacant_pos_list)):
        (x,y) = vacant_pos_list[i]
        child_state = children_states[i]
        child_state.grid_list[x][y] = 'O'

    return children_states

  def input_state_from_min_player(self) -> TicTacToe_Grid_State:
    position = int(input())
    # input is position
    grid_list = copy.deepcopy(self.current_state.grid_list)
    # taking copy of teh current state grid
    while position not in range(self.length_of_tic_tac_toe * self.length_of_tic_tac_toe) or grid_list[int(position/self.length_of_tic_tac_toe)][position%self.length_of_tic_tac_toe] != ' ':
      print('Invalid input')
      # if the input is invalid
      position = (int(input()))
    (x,y) = (int(position/self.length_of_tic_tac_toe),position%self.length_of_tic_tac_toe)

    grid_list[x][y] = 'O'
    new_state = TicTacToe_Grid_State(grid_list)
    # new_state.print_current_grid_state()
    return new_state
    
    


  def start_game_minimax(self):
    # minimax
    self.max_depth_minimax = 0
    self.number_of_leaves_of_the_game_tree_explored = 0
    m = self.length_of_tic_tac_toe
    instruction_list = [
      [i + j*m for i in range(m)] for j in range(m)
    ]
    # instruction list showing the positions of the cells
    instruction_state = TicTacToe_Grid_State(instruction_list)
    instruction_state.print_current_grid_state()
    del instruction_state
    depth = 0



    while not self.isTerminal_state(self.current_state):
      print('Enter postion for O player')
      print()
      print()
      self.current_state = self.input_state_from_min_player()
      # print('the grid after player\'s input is ')
      self.current_state.print_current_grid_state()
      if self.isTerminal_state(self.current_state):
        break
      print()
      print()
      # self.current_state,depth = self.minimax_decision(self.current_state)
      (max_state,depth) = self.minimax_decision(self.current_state)
      
      # to find the depth
      if depth > self.max_depth_minimax:
        self.max_depth_minimax = depth
      max_state: TicTacToe_Grid_State
      self.current_state = max_state
      
      # print(f'The min value is {self.min_value(self.current_state)}')
      self.current_state: TicTacToe_Grid_State
      self.current_state.print_current_grid_state()

    # after the game is over
    final_utility = self.utility_of_terminal_state(self.current_state)
    if final_utility == 0:
      print('the game ends in a draw')
    elif final_utility == 1:
      print('Agent won')
    else:
      print('You won')
    # self.max_depth_minimax = depth
    self.number_of_leaves_of_the_game_tree_explored_for_minimax = self.number_of_leaves_of_the_game_tree_explored
  
  def start_game_alpha_beta(self):
    self.max_depth_alpha_beta = 0
    # alpha beta pruning
    self.current_state = self.initial_state_of_every_game()
    self.number_of_leaves_of_the_game_tree_explored = 0
    m = self.length_of_tic_tac_toe
    instruction_list = [
      [i + j*m for i in range(m)] for j in range(m)
    ]
    instruction_state = TicTacToe_Grid_State(instruction_list)
    instruction_state.print_current_grid_state()
    del instruction_state
    while not self.isTerminal_state(self.current_state):
      print('Enter postion for O player')
      print()
      print()
      self.current_state = self.input_state_from_min_player()
      print('the grid after player\'s input is ')

      self.current_state.print_current_grid_state()
      if self.isTerminal_state(self.current_state):
        # print('Terminal state reached')
        break
      print()
      print()

      max_state, depth = self.alpha_beta_search(self.current_state)
      # alpha beta decision
      max_state: TicTacToe_Grid_State

      if depth > self.max_depth_alpha_beta:
        self.max_depth_alpha_beta = depth


      # self.current_state = self.alpha_beta_search(self.current_state)
      
      self.current_state = max_state
      # changing the current state
      self.current_state.print_current_grid_state()

    # after the game
    final_utility = self.utility_of_terminal_state(self.current_state)
    if final_utility == 0:
      print('the game ends in a draw')
    elif final_utility == 1:
      print('Agent won')
    else:
      print('You won')
    
    self.number_of_leaves_of_the_game_tree_explored_for_alpha_beta = self.number_of_leaves_of_the_game_tree_explored
    


  def isTerminal_state(self, grid_state: TicTacToe_Grid_State,print_grid = False) -> bool:
    m = grid_state.grid_side_length
    # number of leaves explored will increase only when this returns True
    # print(m)
    # check rows
    if(print_grid):
      grid_state.print_current_grid_state()

    for row in grid_state.grid_list:
      if row.count('X') == m or row.count('O') == m:
        self.number_of_leaves_of_the_game_tree_explored += 1
        return True
    
    # column checking
    for i in range(m):
      column = [
        grid_state.grid_list[j][i] for j in range(m)
      ]
      # if print_grid:
      #   print(column)
      if column.count('X') == m or column.count('O') == m:
        self.number_of_leaves_of_the_game_tree_explored += 1
        return True

    # diagonals
    diagonal1 = [
      grid_state.grid_list[i][i] for i in range(m)
    ]
    if diagonal1.count('X') == m or diagonal1.count('O') == m:
      self.number_of_leaves_of_the_game_tree_explored += 1
      return True
    # diagonal1 is a new list
    del diagonal1

    diagonal2 = [
      grid_state.grid_list[i][m-i-1] for i in range(m)
    ]
    if diagonal2.count('X') == m or diagonal2.count('O') == m:
      self.number_of_leaves_of_the_game_tree_explored += 1
      return True
    del diagonal2

    # if the grid is not empty, return False
    for i in range(m):
      for j in range(m):
        if grid_state.grid_list[i][j] == ' ':
          return False
        

    return True
  

  def utility_of_terminal_state(self, grid_state: TicTacToe_Grid_State) -> int:
    m = grid_state.grid_side_length
    # print(m)
    # check rows
    for row in grid_state.grid_list:
      if row.count('X') == m:
        return 1
      elif row.count('O') == m:
        return -1
    
    # column checking
    for i in range(m):
      column = [
        grid_state.grid_list[j][i] for j in range(m)
      ]
      if column.count('X') == m:
        return 1
      elif column.count('O') == m:
        return -1
      

    # diagonals
    diagonal1 = [
      grid_state.grid_list[i][i] for i in range(m)
    ]
    if diagonal1.count('X') == m:
      return 1
    elif diagonal1.count('O') == m:
      return -1
    # diagonal1 is a new list
    del diagonal1

    diagonal2 = [
      grid_state.grid_list[i][m-i-1] for i in range(m)
    ]
    if diagonal2.count('X') == m:
      return 1
    elif diagonal2.count('O') == m:
      return -1
    del diagonal2

    return 0
  
  def minimax_decision(self, grid_state: TicTacToe_Grid_State) -> tuple:
    self.depth_count = 0
    children_states = self.child_generator(grid_state,current_player_max=True)
    # children_states_utility_values = [
    #   self.min_value(child_state) for child_state in children_states
    # ]
    children_states_utility_values = []
    depth_list = []

    for i in range(len(children_states)):
      child_state = children_states[i]
      # min value returns the utility and the max depth
      utility_to_be_appended, depth = self.min_value(child_state)
      children_states_utility_values.append(utility_to_be_appended)
      depth_list.append(depth)
    
    max_value_of_utility = max(children_states_utility_values)
    for i in range(len(children_states)):
      if children_states_utility_values[i] == max_value_of_utility:
        depth_max = max(depth_list)
        del depth_list
        # return the action which maximises the min value of children states
        return children_states[i],depth_max


  

  def min_value(self, grid_state: TicTacToe_Grid_State) -> tuple:
    # print('inside max_value')
    # self.depth_count +=1 
    if self.isTerminal_state(grid_state):
      return self.utility_of_terminal_state(grid_state),1
    
    v = 10
    # simulating positive infinity
    depth_list = []
    # to chose the maximum depth

    children_states = self.child_generator(grid_state,current_player_max = False)
    # children_states = self.child_generator(grid_state,current_player_max = True)
    for child_state in children_states:
      # print('in max-value')
      # child_state.print_current_grid_state()

      current_utility, depth = self.max_value(child_state)

      # v,depth = min(v, self.max_value(child_state))
      v = min(v,current_utility)
      depth_list.append(1 + depth)
    del children_states
    depth_max = max(depth_list)
    del depth_list

    return v,depth_max

  def max_value(self, grid_state: TicTacToe_Grid_State) -> tuple:
    # self.depth_count = 
    # print('inside min_value')
    # print('of')
    # grid_state.print_current_grid_state()
    if self.isTerminal_state(grid_state):
      return self.utility_of_terminal_state(grid_state),1
    
    v = -10
    # simulating negative infinity
    children_states = self.child_generator(grid_state,current_player_max = True)
    # children_states = self.child_generator(grid_state,current_player_max = False)
    depth_list = []
    for child_state in children_states:
      # print('in min-value')
      # child_state.print_current_grid_state()

      current_utility, depth = self.min_value(child_state)
      # (v,depth) = max(v, self.min_value(child_state))
      v = max(v,current_utility)
      depth_list.append(1 + depth)

    del children_states
    depth_max = max(depth_list)
    del depth_list


    return v,depth_max




  def alpha_beta_search(self, grid_state: TicTacToe_Grid_State) -> TicTacToe_Grid_State:
    # self.alpha = -10
    # self.beta = 10
    alpha=-1
    beta=1
    self.depth_count = 0
    children_states = self.child_generator(grid_state,current_player_max=True)

    v = -1
    
    # children_states_utility_values = [
    #   self.min_value_alpha_beta(child_state,alpha,beta) for child_state in children_states
    # ]

    utiltity_values = []

    depth_list = []
    depth_max = 0
    for i in range(len(children_states)):
      current, depth = self.min_value_alpha_beta(children_states[i],alpha,beta)

      depth_list.append(depth)

      v = max(v, current)
      utiltity_values.append(v)
      # if v greater than the alpha, update alpha
      if v > alpha:
        alpha = v
      if current > beta:
        # if the current is greater than the beta which was seen previously, then no need of 
        # checking the remaining branches because min will anyways won't take this path
        
        depth_max = max(depth_list)
        del depth_list
        return children_states[i-1], depth_max
      

    if depth_list:
      depth_max = max(depth_list)
      del depth_list
    
    utiltity_max = max(utiltity_values)
    for i in range(len(utiltity_values)):
      if utiltity_values[i] == utiltity_max:
        return children_states[i],depth_max



  def max_value_alpha_beta(self, grid_state: TicTacToe_Grid_State,alpha,beta) -> int:
    if self.isTerminal_state(grid_state):
      return self.utility_of_terminal_state(grid_state),1
    v = -10
    # simulating negative infinity
    children_states = self.child_generator(grid_state,current_player_max = True)

    depth_list = []
    depth_max = 0

    for child_state in children_states:
      current,depth = self.min_value_alpha_beta(child_state,alpha,beta)
      depth_list.append(1 + depth)

      # current level + depth seen so far
      # that is 1+depth
      v = max(v, current)

      if v > alpha:
        # if v greater than alpha, update v
        alpha = v
      if current > beta:
        # if current is greater than beta, then we can prune the other branches, because
        # min player has a better path anyways
        depth_max = max(depth_list)
        del depth_list


        del children_states

        return current, depth_max

    if children_states:
      del children_states
    
    if depth_list:
      depth_max = max(depth_list)
      del depth_list

    return v,depth_max
  
  def min_value_alpha_beta(self, grid_state: TicTacToe_Grid_State,alpha,beta) -> int:
    # self.depth_count = 
    # print('inside min_value')

    if self.isTerminal_state(grid_state):
      return self.utility_of_terminal_state(grid_state),1
    v = 10
    # simulating positive infinity
    children_states = self.child_generator(grid_state,current_player_max = False)

    depth_list = []
    depth_max = 0
    for child_state in children_states:
      current, depth =self.max_value_alpha_beta(child_state,alpha,beta)
      depth_list.append(1 + depth)
      v = min(v, current)
      if v<beta:
        # if v less than beta, update beta
        beta=v
      if current<alpha:
        # if current utility is less than the alpha seen so far, then we can prune the other branches because max player already has a better path
        del children_states

        depth_max = max(depth_list)
        del depth_list
        return current,depth_max
    
    if depth_list:
      depth_max = max(depth_list)
      del depth_list

    if children_states:
      del children_states
    return v, depth_max


(a) Use the minimax strategy to design an AI that plays the game optimally. The leaf nodes where "X" wins gets 1 and "O" wins gets -1 and neither wins gets a zero. A sample game play is given below.

In [24]:
agent = Agent()
agent.number_of_leaves_of_the_game_tree_explored = 0
agent.start_game_minimax()

minimax_explored_leaves = agent.number_of_leaves_of_the_game_tree_explored



0|1|2
-+-+-
3|4|5
-+-+-
6|7|8
Enter postion for O player


 | | 
-+-+-
 |O| 
-+-+-
 | | 


X| | 
-+-+-
 |O| 
-+-+-
 | | 
Enter postion for O player


X| | 
-+-+-
 |O| 
-+-+-
O| | 


X| |X
-+-+-
 |O| 
-+-+-
O| | 
Enter postion for O player


X|O|X
-+-+-
 |O| 
-+-+-
O| | 


X|O|X
-+-+-
 |O| 
-+-+-
O|X| 
Enter postion for O player


X|O|X
-+-+-
O|O| 
-+-+-
O|X| 


X|O|X
-+-+-
O|O|X
-+-+-
O|X| 
Enter postion for O player


X|O|X
-+-+-
O|O|X
-+-+-
O|X|O
the game ends in a draw


(b) Modify the previous answer to calculate in each game the following:

* the maximum depth of exploration of the game tree in a game, and
* number of leaves of the game tree whose scores were computed be the end of the game.




In [25]:
print(f'the number of leaves of game tree explored in minimax is {agent.number_of_leaves_of_the_game_tree_explored}')
print()
print(f'the maximum depth is {agent.max_depth_minimax}')

the number of leaves of game tree explored in minimax is 21647

the maximum depth is 8


(c) Implement alpha-beta pruning minimax search to solve the Tic-Tac-Toe and repeat part (b). 

Compare your results with vanilla minimax search and see on which all parameters part (b) is alpha-bet pruning search better. Also obtain by what factor is it better.

In [26]:
agent.start_game_alpha_beta()


0|1|2
-+-+-
3|4|5
-+-+-
6|7|8
Enter postion for O player


the grid after player's input is 
 | | 
-+-+-
 |O| 
-+-+-
 | | 


X| | 
-+-+-
 |O| 
-+-+-
 | | 
Enter postion for O player


the grid after player's input is 
X| | 
-+-+-
 |O| 
-+-+-
O| | 


X| |X
-+-+-
 |O| 
-+-+-
O| | 
Enter postion for O player


the grid after player's input is 
X|O|X
-+-+-
 |O| 
-+-+-
O| | 


X|O|X
-+-+-
 |O| 
-+-+-
O|X| 
Enter postion for O player


the grid after player's input is 
X|O|X
-+-+-
O|O| 
-+-+-
O|X| 


X|O|X
-+-+-
O|O|X
-+-+-
O|X| 
Enter postion for O player


the grid after player's input is 
X|O|X
-+-+-
O|O|X
-+-+-
O|X|O
the game ends in a draw


In [27]:
print(f'the number of leaves of game tree in alpha beta explored is {agent.number_of_leaves_of_the_game_tree_explored}')
print()
print(f'the maximum depth is {agent.max_depth_alpha_beta}')


alpha_beta_leaves = agent.number_of_leaves_of_the_game_tree_explored

the number of leaves of game tree in alpha beta explored is 3265

the maximum depth is 8


In [28]:
print(f'ALPHA BETA PRUNING searches {alpha_beta_leaves} leaves whereas MINIMAX searches {minimax_explored_leaves} leaves')

print(f'Alpha Beta pruning is {minimax_explored_leaves/alpha_beta_leaves} times efficient than minimax')

ALPHA BETA PRUNING searches 3265 leaves whereas MINIMAX searches 21647 leaves
Alpha Beta pruning is 6.630015313935681 times efficient than minimax
